# Chains
A building block that combines LLM with a prompt, or allows combining multiple such building blocks. Chains are responsible for creating an end-to-end pipeline for using the language models. They will join the model, prompt, memory, parsing output, and debugging capability and provide an easy-to-use interface.

- LLMChain
- ConversationalChain
- Sequential Chains
    - SimpleSequentialChain
    - SequentialChain
- Router Chain
- Custom Chain
---

## Setup

In [1]:
import openai
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
openai.api_type = os.environ.get("OPENAI_API_TYPE")
openai.api_base = os.environ.get("OPENAI_API_BASE")
openai.api_key = os.environ.get("OPENAI_API_KEY")
openai.api_version = os.environ.get("OPENAI_API_VERSION")

## LLMChain
Most basic chain, combines LLM with a prompt.

In [10]:
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [3]:
llm = AzureChatOpenAI(
    deployment_name="gpt4",
    temperature=0.9,
)

In [4]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

chain = LLMChain(llm=llm, prompt=prompt)

### Ways to call chain

#### 1. call via `run()` function
It takes input variables and returns the output of the language model.

In [5]:
product = "Soft Toys"
response = chain.run(product=product)
response

'Cuddle Creations'

#### 2. call via `predict()` function
Similar to `run()` function, takes input variables and returns the output of the language model.

In [6]:
product = "Soft Toys"
response = chain.predict(product=product)
response

'Cuddle Creations'

#### 3. directly pass a prompt
Pass the input variables directly to the chain, returns a dictionary containing all input variables and output in the `text` key. 
> Note: Only works with one input variable.

In [7]:
product = "Soft Toys"
response = chain(product)
response

{'product': 'Soft Toys', 'text': 'Cuddle Creations'}

#### 4. call via `apply()` function
It takes multiple inputs at once, and return a list of dicts where each dict has output in the key `text`. The returned list will maintain the identical order as the input.

In [8]:
products = [
    {"product": "Soft Toys"},
    {"product": "Legos"},
    {"product": "Cars"},
]
response = chain.apply(products)
response

[{'text': 'Cuddly Creations'},
 {'text': 'BrickBuilders'},
 {'text': 'AutoCrafters'}]

#### 5. call via `generate()` function
It takes multiple inputs at once, and return an instance of `LLMResult`, which provides more information. For example, the `finish_reason` key indicates the reason behind the stop of the generation process. It could be stopped, meaning the model decided to finish or reach the length limit. There is other self-explanatory information like the number of total used tokens or the used model.

In [9]:
response = chain.generate(products)
response

LLMResult(generations=[[ChatGeneration(text='Cuddly Creations Toys', generation_info=None, message=AIMessage(content='Cuddly Creations Toys', additional_kwargs={}, example=False))], [ChatGeneration(text='Brick Builders', generation_info=None, message=AIMessage(content='Brick Builders', additional_kwargs={}, example=False))], [ChatGeneration(text='Auto Innovations', generation_info=None, message=AIMessage(content='Auto Innovations', additional_kwargs={}, example=False))]], llm_output={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 65, 'total_tokens': 77}, 'model_name': 'gpt-3.5-turbo'}, run=[RunInfo(run_id=UUID('0a99a8d6-cda0-40b5-b10a-08c8c8088b8b')), RunInfo(run_id=UUID('b2ecdce9-1c8f-4656-8ad9-db1a0d7f287a')), RunInfo(run_id=UUID('441876ce-9e02-4f46-a565-971f32d69ffd'))])

### Directly pass a prompt as string to a `Chain` and initialize it using `from_string()`

In [16]:
# template = prompt.messages[0].prompt.template # this is the same as line below
template = "What is the best name to describe a company that makes {product}?"

chain = LLMChain.from_string(llm=llm, template=template)

product = "Soft Toys"
response = chain.run(product=product)
response

'Cuddle Creations'

### Parsers in LLMChain
It is essential to use the `.predict_and_parse()` method to get a response with the List data type. Otherwise, the `.predict()` function will return the unchanged model response.

In [20]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate

output_parser = CommaSeparatedListOutputParser()
template = """List all possible words as substitute for '{word}' as comma separated."""

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(
        template=template, output_parser=output_parser, input_variables=["word"]
    ),
)

llm_chain.predict_and_parse(word="artificial")

['synthetic',
 'man-made',
 'imitation',
 'simulated',
 'fake',
 'faux',
 'ersatz',
 'counterfeit',
 'fabricated',
 'manufactured',
 'unnatural',
 'lab-created']

## ConversationalChain (Memory)

In [23]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
conversation = ConversationChain(llm=llm, memory=memory, verbose=True)
conversation.predict(input="Hi, my name is Alice?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Alice?
AI:

> Finished chain.


"Hello Alice! It's nice to meet you. I'm an AI here to help with any questions you may have or simply to have a nice conversation. How can I assist you today?"

In [24]:
conversation.predict(input="What's 1+1?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Alice?
AI: Hello Alice! It's nice to meet you. I'm an AI here to help with any questions you may have or simply to have a nice conversation. How can I assist you today?
Human: What's 1+1?
AI:

> Finished chain.


'1+1 is equal to 2.'

In [25]:
conversation.predict(input="What's my name?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Alice?
AI: Hello Alice! It's nice to meet you. I'm an AI here to help with any questions you may have or simply to have a nice conversation. How can I assist you today?
Human: What's 1+1?
AI: 1+1 is equal to 2.
Human: What's my name?
AI:

> Finished chain.


'Your name is Alice.'

## Sequential Chains
- Combine multiple chains where the output of one chain is input of next chain.
- Two types:
    - `SimpleSequentialChain`: Single input/output
    - `SequentialChain`: Multiple inputs/outputs

### SimpleSequentialChain

In [6]:
from langchain.chains import SimpleSequentialChain

In [7]:
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [8]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [9]:
overall_simple_chain = SimpleSequentialChain(
    chains=[chain_one, chain_two], verbose=True
)
response = overall_simple_chain.run(product)
response



> Entering new  chain...
Cuddly Creations Soft Toys
Cuddly Creations Soft Toys designs and produces irresistibly adorable, high-quality plush companions for children and collectors, sparking joy and comfort in every snuggle.

> Finished chain.


'Cuddly Creations Soft Toys designs and produces irresistibly adorable, high-quality plush companions for children and collectors, sparking joy and comfort in every snuggle.'

### SequentialChain

In [10]:
from langchain.chains import SequentialChain

In [11]:
# prompt template 1: translate original review to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:" "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="English_Review")

In [12]:
# prompt template 2: summarize translated review
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:" "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")

In [13]:
# prompt template 3: detect language of original review
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="language")

In [14]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key="followup_message")

In [15]:
# overall_chain: input= Review
# and output= English_Review, summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary", "followup_message"],
    verbose=True,
)

In [16]:
review = "Howdy folks, I am a big fan of this product. I have been using it for 2 years now and I am very happy with it."
response = overall_chain(review)
response



> Entering new  chain...

> Finished chain.


{'Review': 'Howdy folks, I am a big fan of this product. I have been using it for 2 years now and I am very happy with it.',
 'English_Review': 'Hello everyone, I am a huge fan of this product. I have been using it for 2 years now and I am extremely satisfied with it.',
 'summary': 'The reviewer is a huge fan of the product, having used and been extremely satisfied with it for two years.',
 'followup_message': 'It is fantastic to hear that you have been a dedicated user of our product for the past two years and have been extremely satisfied with it. Knowing that we have such loyal fans means a lot to us, and your positive feedback is greatly appreciated. We will continue to strive for excellence and ensure that our product maintains the high quality that you have come to expect. Thank you once again for sharing your experience, and we look forward to serving you for many more years to come.'}

In [17]:
type(response)

dict

## Router Chain
Allows to route an input to a chain depending on what exactly that input is.

In [18]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [19]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template,
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template,
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template,
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template,
    },
]

In [20]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

In [21]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [22]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [23]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [24]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [29]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [30]:
response = chain.run("What is black body radiation?")
response



> Entering new  chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation is the type of electromagnetic radiation emitted by a perfect black body, which is an idealized object that absorbs all incoming light and does not reflect any. This radiation is a result of the thermal energy of the object, and its characteristics depend only on the temperature of the object. The spectrum of black body radiation, described by Planck's Law, is continuous and covers a wide range of wavelengths. As the temperature of the black body increases, the peak of the spectrum shifts towards shorter wavelengths, following Wien's Displacement Law, and the overall intensity increases as described by the Stefan-Boltzmann Law. Black body radiation is crucial for understanding various physical phenomena, including the temperature of stars and the cosmic microwave background radiation."

In [27]:
response = chain.run("what is 2 + 2")
response



> Entering new  chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'To solve this problem, we can simply combine the two numbers by performing addition:\n\n2 + 2 = 4\n\nSo, the answer to the question is 4.'

In [28]:
response = chain.run("Why does every cell in our body contain DNA?")
response



> Entering new  chain...
None: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA is the genetic blueprint that carries the instructions for the structure, function, growth, and reproduction of an organism. It is crucial for encoding the proteins and molecules required for cellular processes and maintaining the overall health and function of the organism.\n\nIn multicellular organisms like humans, DNA is inherited from both parents and is present in the nucleus of each cell. This genetic material ensures that each cell of the body can perform its specific function, and it also enables the process of cell division, allowing cells to replicate and replace damaged or old cells. Additionally, DNA provides the basis for individual traits and characteristics, making each person unique.\n\nHowever, it is worth noting that not all cells have the same active genes, even though they all contain the same DNA. Different cell types, such as muscle cells, skin cells, or nerve cells, have specific functions, and the required genes f

## Custom Chain
It is also possible to define your chain for any custom task. In this section, we will create a chain that returns a word's meaning and then suggests a replacement.

It starts by defining a class that inherits most of its functionalities from the `Chain` class. Then, the following three methods must be declared depending on the use case. The `input_keys` and `output_keys` methods let the model know what it should expect, and a `_call` method runs each chain and merges their outputs.

In [26]:
from langchain.chains import LLMChain
from langchain.chains.base import Chain

from typing import Dict, List


class ConcatenateChain(Chain):
    chain_1: LLMChain
    chain_2: LLMChain

    @property
    def input_keys(self) -> List[str]:
        # Union of the input keys of the two chains.
        all_input_vars = set(self.chain_1.input_keys).union(
            set(self.chain_2.input_keys)
        )
        return list(all_input_vars)

    @property
    def output_keys(self) -> List[str]:
        return ["concat_output"]

    def _call(self, inputs: Dict[str, str]) -> Dict[str, str]:
        output_1 = self.chain_1.run(inputs)
        output_2 = self.chain_2.run(inputs)
        return {"concat_output": output_1 + output_2}

In [28]:
prompt_1 = PromptTemplate(
    input_variables=["word"],
    template="What is the meaning of the following word '{word}'?",
)
chain_1 = LLMChain(llm=llm, prompt=prompt_1)

prompt_2 = PromptTemplate(
    input_variables=["word"],
    template="What is a word to replace the following: {word}?",
)
chain_2 = LLMChain(llm=llm, prompt=prompt_2)

concat_chain = ConcatenateChain(chain_1=chain_1, chain_2=chain_2)
concat_output = concat_chain.run("artificial")
concat_output

'The word "artificial" refers to something that is made or produced by human beings rather than occurring naturally. It typically implies imitation or replication of a natural object or process, often with the intent to improve upon or control specific aspects of the original.Synthetic'

In [29]:
# Another way to call the chain
concat_chain("artificial")

{'word': 'artificial',
 'concat_output': 'The word "artificial" refers to something that is made or produced by human beings rather than occurring naturally. It often implies imitation or replication of a natural object or process, and may also suggest that the artificial object is inferior or less genuine than the natural one.Synthetic'}